# Wikipedia data extraction: Spotify streaming records

Use requests and beautiful soup to download and parse spotify streaming records from Wikipedia.

## 1. Notebook set-up

In [1]:
# External libraries 
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
# Target URL
url='https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records'

# Web browser-like request headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36' +
        '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

## 2. Download the data

In [3]:
# Catch any errors with try-except block
try:
    # Fetch the book page
    response = requests.get(url, headers=headers, timeout=15)
    response.raise_for_status()

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

except requests.exceptions.RequestException as e:
    print(f'Caught exception from requests: {e}')
    soup = None

## 3. Parse the data

### 3.1. Get the table rows

In [4]:
tables = soup.find_all('table')
most_streamed_table = tables[0]
column_names = most_streamed_table.find_all('tr')[0]

### 3.2. Get the column names

In [5]:
# Get just the header row from the table
table_header = most_streamed_table.find_all('tr')[0]

# Loop on the th tags to collect the column names
column_names = []

for column in table_header.find_all('th'):
    column_names.append(column.get_text().strip())

print(f'Column names: {column_names}')

Column names: ['Rank', 'Song', 'Artist(s)', 'Streams(billions)', 'Release date', 'Ref.']



### 3.3. Get the data from the table body

In [6]:
# List
results = []

# Get the rows of data from the table body
data_rows = most_streamed_table.find_all('tr')[1:]

# Loop on rows and extract the value from each column
for data_row in data_rows:

    # Empty list to hold the values for this row
    values = []

    # Loop on the tags in this row
    for tag in data_row:

        # Song titles are nested inside of a <th> and an <a> tag.
        if tag.name == 'th':
            
            link = None
            link = tag.find('a')

            if link:
                title = link.get_text().strip()
            else:
                title = None
            
            values.append(title)

        # Everything else is directly inside of the top-level <td>
        elif tag.name == 'td':

            values.append(tag.get_text().strip())
    
    results.append(values)

### 3.4. Convert to Pandas DataFrame

In [7]:
results_df = pd.DataFrame(results, columns=column_names)
results_df.head()

,Rank,Song,Artist(s),Streams(billions),Release date,Ref.
0,1,Blinding Lights,The Weeknd,4.914,29 November 2019,[2]
1,2,Shape of You,Ed Sheeran,4.441,6 January 2017,[3]
2,3,Starboy,The Weeknd and Daft Punk,3.975,21 September 2016,[4]
3,4,Someone You Loved,Lewis Capaldi,3.962,8 November 2018,[5]
4,5,As It Was,Harry Styles,3.953,1 April 2022,[6]
